# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** <Write your name and Roll number here>
    

In [1]:
import os 
import random
import time
from collections import deque
import datetime
from urllib.parse import urlparse
from urllib import robotparser
import requests
from bs4 import BeautifulSoup
import threading
# Add any library to be imported here

In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
S = threading.Semaphore(1)
# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [3]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
    def __init__(self):
        self.fQs = [deque() for i in range(FRONTQUEUES)]
        self.bQs = [{"Q":deque(),"t":datetime.datetime.now(),"d":''} for j in range(BACKQUEUES)]
        '''
        Initializer
        '''
        None # replace this with your code
        
    # add more functions here
    def addSeed(self,urls):
        for url in urls:
            self.add_to_fQ(url)
    def add_to_fQ(self,url):
        pr = prioritizer(url,FRONTQUEUES)
        self.fQs[pr-1].append(url)
    def move_to_bQ(self):
        pr = random.randint(1,FRONTQUEUES)
        if self.fQs[pr-1]:
            url = self.fQs[pr-1].popleft()
            domain = urlparse(url).netloc
            ind = [i for i in range(BACKQUEUES) if domain in self.bQs[i]["d"]]
            if len(ind)>0:
                self.bQs[ind[0]]["Q"].append(url)
            else:
                ind = [i for i in range(BACKQUEUES) if not self.bQs[i]["Q"]]
                if len(ind)>0:
                    self.bQs[ind[0]]["d"]=domain
                    self.bQs[ind[0]]["t"]=datetime.datetime.now()
                    self.bQs[ind[0]]["Q"].append(url)
    def get_url_from_bQ(self):
        sortedQs = sorted(self.bQs,key=lambda x: x['t'], reverse=True)
        if sortedQs[0]['Q']:
            wait = 0
            c_t = datetime.datetime.now()
            if sortedQs[0]['t'] > c_t:
                wait=sortedQs[0]['t']-c_t
                print(" wait:"+str(wait.seconds)+"seconds \n")
                time.sleep(wait.seconds)
            #if wait.seconds > 0:
             #   time.sleep(wait.seconds)
            S.acquire()
            if sortedQs[0]['Q']:
                sortedQs[0]['d'] = ' '
                url=sortedQs[0]['Q'].popleft()
                for i in range(BACKQUEUES):
                    d = urlparse(url).netloc
                    if d in self.bQs[i]['d']:
                        self.bQs[i]['t'] = datetime.datetime.now() + datetime.timedelta(seconds=WAITTIME)
                        break
                S.release()
                return url
            S.release()
        return None
        #print(sortedQs)

def prioritizer(URL,f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function. 
    It will return a random number from 1 to f for given inputs. 
    """
    return random.randint(1,f)
    

# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [4]:
# write code for filtering urls here 
def filter_url(url):
    domain = urlparse(url).netloc
    try:
        rp = urllib.robotparser.RobotFileParser()
        rp.set_url(urlparse(url).scheme+"://"+domain+'/robots.txt')
        rp.read()
        return not rp.can_fetch('*',url)
    except:
        return False

## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [5]:
# Theard task
def crawler_thread_task(frontier):
    while len(fetched_links) <= 1000:
        url = frontier.get_url_from_bQ()
        S.release()
        if url and url not in fetched_links:
            print(" processing: "+url)
            fetched_links.append(url)
            domain = urlparse(url).netloc
            scheme = urlparse(url).scheme
            req = requests.get(url, headers)
            soup = BeautifulSoup(req.content, 'html.parser')
            urls = soup.find_all('a',href=True)
            urls=[url['href'] for url in urls] 
            for url in urls:
                if 'http' in url:
                    try:
                        r = requests.head(url)
                        if "text/html" in r.headers["content-type"]:
                            if filter_url(url) == False:
                                frontier.add_to_fQ(url)
                    except:
                        pass
                elif url[0] == '/':
                    try:
                        r = requests.head(scheme+"://"+domain+url)
                        if "text/html" in r.headers["content-type"]:
                            if filter_url(scheme+"://"+domain+url) == False:
                                frontier.add_to_fQ(scheme+"://"+domain+url)
                    except:
                        pass
        frontier.move_to_bQ()
    print("Crwaler Stopped\n")
    None # define individual crawler thread's function here as studies in class

In [6]:
fetched_links =[]
f = frontier()
f.addSeed(["https://docs.oracle.com/en/","https://www.oracle.com/corporate/","https://en.wikipedia.org/wiki/Machine_learning","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html","https://en.wikipedia.org/w/api.php","https://en.wikipedia.org/api/","https://en.wikipedia.org/wiki/Weka_(machine_learning)"])

In [ ]:
# start the threads
ts=[]
for i in range(THREADS):
    t = threading.Thread(target=crawler_thread_task, args=(f,))
    ts.append(t)
    t.start()
while 1:
    pass

 processing: https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html
 processing: https://en.wikipedia.org/wiki/Machine_learning
 processing: https://en.wikipedia.org/w/api.php
 processing: https://docs.oracle.com/en/
 processing: https://www.oracle.com/corporate/
 processing: https://en.wikipedia.org/api/
 processing: https://en.wikipedia.org/wiki/Weka_(machine_learning)
 processing: https://www.oracle.com/corporate/accessibility/
 processing: http://www.gnuplot.info/
 processing: https://en.wikipedia.org/wiki/Weka_(disambiguation)
 processing: https://foundation.wikimedia.org/wiki/Developer_app_guidelines
 processing: https://www.mediawiki.org/wiki/Special:MyLanguage/API:Main_page
 processing: https://en.wiktionary.org/wiki/weka
 processing: https://en.wikipedia.org/wiki/File:Weka_(software)_logo.png
 processing: https://en.wikipedia.org/wiki/Weka
 processing: https://en.wikipedia.org/wiki/Special:Version/License/MediaWiki
 processing: http://clopinet.com/isabelle/Projects/NIPS2003/
 pr

## ------------------------------------------------------End of Notebook---------------------------------------------------